# 目錄



## [STEP05 組合 STEP01 - STEP04 ](#STEP4)


### &emsp;[讀檔 & 切割](#讀檔)


### &emsp;[Target Table 部分 - 01. Drop Table](#Drop-Table)


### &emsp;[Target Table 部分 - 02. Create Table](#Create-Table)


### &emsp;[Target Table 部分 - 03. Delete](#Delete)


### &emsp;[Target Table 部分 - 04. Insert & 其他](#Insert-Into)


### &emsp;[Source Table 部分 - 01. From](#From)


### &emsp;[Source Table 部分 - 02. Join](#Join)


### &emsp;[組合](#組合)


### &emsp;[重新整理](#重新整理-Report-List)


### &emsp; &emsp;[EX. 去重複](#去重複)


### &emsp;[最後的整理](#最後整合)


### &emsp;[批次處理所有檔案](#大量做)










# Import Package

In [ ]:
import pathlib
import re
import pandas as pd
import time

from IPython.display import clear_output, display


from itertools import zip_longest

# Remove Pandas DataFrame Default Truncate Value
pd.set_option('display.max_colwidth', None)

# Script File 在 本機的位置

In [ ]:

# Script 檔案所在資料夾
path = 'Script/DWSCRIPT/'

# 要抓取的檔案 全部都是 .txt 為副檔名
# File Pattern 
file_pattern = '*.txt'

p = pathlib.Path(path)

p.exists()

# 抓檔

In [ ]:

# 路徑確定正確後
# 要來抓檔了

# 以 副檔名皆為 .txt 來抓取
files = p.glob(file_pattern)

files = [f for f in files]

for f in files:
    print(f)

In [ ]:
for i, f in enumerate(files):
    
    if f.name == 'EDW_PROD REPORT_MONTHLY SR233168_VIP_GEN STEP0100 sql.txt':
        print(i)

# 讀檔


## 1. 避免 Encoding 問題


## 2. 先把檔案切成一行一行


[目錄](#目錄)

In [ ]:

def readline_file_with_encode(file):
    
    # Return List
    
    try:
        with open(file, 'r', encoding='UTF-8') as f:
            
            content = f.read()
            
    except UnicodeDecodeError:
        with open(file, 'r', encoding='MS950') as f:
            content = f.read()

    return content.split('\n')


# STEP4


## 組合一下 STEP1 ~ STEP3


## 同時產生 Report DataFrame


[目錄](#目錄)

### 等等要用的 預備 Column Names



### 我們等等要用的 欄位如下


|  檔案名稱  |  Operator  | Source/Target Line  | Source/Target Table 
|  ----  | ----  |  ----  | ----
| File1  | FROM | 4 | PDATA.TABLENAME 
| File1  | JOIN | 9 | PMART.TABLENAME 


In [ ]:
target_column = ['FileName', 'Operator', 'Target_Lines', 'Target_Table']

source_column = ['FileName', 'Operator', 'Source_Lines', 'Source_Table']

# Target Table

## Drop Table

In [ ]:
# Report list
report = []

# 遞迴所有的 File
for file in files:
    # 現在處理哪個 File
    print('======= 處理 {}========'.format(file.name))
    
    # 讀取檔案
    
    sql_line = readline_file_with_encode(file)
    
    # Step02 - Step03 SQL 邏輯檢查
    for i, line in enumerate(sql_line):
        
        # Target 邏輯盤點
        
        # 1. Drop Table
        if re.search(r'\bDROP TABLE IF EXISTS\b', line, flags=re.I):
            # 去除前置空白 以及 DROP TABLE IF EXISTS 前的多餘字串
            mess = line[line.find("DROP TABLE IF EXISTS"):].replace(";","").lstrip()
            
            if len(mess.split()) <= 4:
                # 以空白切割字串 
                # 若只有 DROP TABLE IF EXISTS 那就只有 4 個字 故長度為 4
                continue
                
            elif len(mess.split()) > 4:
                print("TARGET:DROP TABLE IF EXISTS : " + mess.split()[4].strip())
                # 第 5 個字詞位置 為 Table Name
                
                # 產生 report row
                row = [file.name, 'TARGET:DROP TABLE IF EXISTS', i + 1, mess.split()[4].strip()]
                
                # 加入 report list 中
                report.append(row)
            else:
                continue
                
        elif re.search(r'\bDROP TABLE\b', line, flags=re.I):
            mess = line[line.find("DROP TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:DROP TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:DROP TABLE", i + 1, mess.split()[2].strip()]
                
                report.append(row)
            else:
                continue
        

time.sleep(5)
clear_output()
# 帶入 Target Columns
pd.DataFrame(report, columns=target_column)
    

# Target Table

## Create Table

[目錄](#目錄)

In [ ]:
# Report list
report = []

for file in files:

    print('======= 處理 {}========'.format(file.name))
    
    sql_line = readline_file_with_encode(file)
    
    # Step02 - Step03 SQL 邏輯檢查
    for i, line in enumerate(sql_line):
        
        # Target 邏輯盤點
        
        # 1. Drop Table
        if re.search(r'\bDROP TABLE IF EXISTS\b', line, flags=re.I):
            # 去除前置空白 以及 DROP TABLE IF EXISTS 前的多餘字串
            mess = line[line.find("DROP TABLE IF EXISTS"):].replace(";","").lstrip()
            
            if len(mess.split()) <= 4:
                # 以空白切割字串 
                # 若只有 DROP TABLE IF EXISTS 那就只有 4 個字 故長度為 4
                continue
                
            elif len(mess.split()) > 4:
                print("TARGET:DROP TABLE IF EXISTS : " + mess.split()[4].strip())
                # 第 5 個字詞位置 為 Table Name
                row = [file.name, "TARGET:DROP TABLE IF EXISTS", i + 1, mess.split()[4].strip()]
                report.append(row)
                
            else:
                continue
                
        elif re.search(r'\bDROP TABLE\b', line, flags=re.I):
            mess = line[line.find("DROP TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:DROP TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:DROP TABLE", i + 1, mess.split()[2].strip()]
                report.append(row)
            else:
                continue
                  
        # 2. Create Table
        elif re.search(r'\bCREATE TABLE\b', line, flags=re.I):
            mess = line[line.find("CREATE TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:CREATE TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:CREATE TABLE", i + 1, mess.split()[2].strip()]
                report.append(row)
            else:
                continue

                
time.sleep(5)
clear_output()

pd.DataFrame(report, columns=target_column)

# Target Table

## Delete

[目錄](#目錄)

In [ ]:
# Report list
report = []

for file in files:

    print('======= 處理 {}========'.format(file.name))
    
    sql_line = readline_file_with_encode(file)
    
    # Step02 - Step03 SQL 邏輯檢查
    for i, line in enumerate(sql_line):
        
        # Target 邏輯盤點
        
        # 1. Drop Table
        if re.search(r'\bDROP TABLE IF EXISTS\b', line, flags=re.I):
            # 去除前置空白 以及 DROP TABLE IF EXISTS 前的多餘字串
            mess = line[line.find("DROP TABLE IF EXISTS"):].replace(";","").lstrip()
            
            if len(mess.split()) <= 4:
                # 以空白切割字串 
                # 若只有 DROP TABLE IF EXISTS 那就只有 4 個字 故長度為 4
                continue
                
            elif len(mess.split()) > 4:
                print("TARGET:DROP TABLE IF EXISTS : " + mess.split()[4].strip())
                # 第 5 個字詞位置 為 Table Name
                row = [file.name, "TARGET:DROP TABLE IF EXISTS", i + 1, mess.split()[4].strip()]
                report.append(row)
                
            else:
                continue
                
        elif re.search(r'\bDROP TABLE\b', line, flags=re.I):
            mess = line[line.find("DROP TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:DROP TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:DROP TABLE", i + 1, mess.split()[2].strip()]
                report.append(row)
            else:
                continue
                  
        # 2. Create Table
        elif re.search(r'\bCREATE TABLE\b', line, flags=re.I):
            mess = line[line.find("CREATE TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
#                 print("TARGET:CREATE TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:CREATE TABLE", i + 1, mess.split()[2].strip()]
                report.append(row)
            else:
                continue

        # 3. Delete Table
        # 要注意的是 DELETE FROM 要先判斷
        elif re.search(r'\bDELETE FROM\b', line, flags=re.I):
            mess = line[line.find("DELETE FROM"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:DELETE FROM : " + mess.split()[2].strip())
                row = [file.name, "TARGET:DELETE FROM", i + 1, mess.split()[2].strip()]
                report.append(row)
        
        #  之後才判斷 DELETE TABLE NOT FROM (TD Only)
        elif re.search(r'\bDELETE\b', line):
            mess = line[line.find("DELETE"):].replace(";","").lstrip()
            if len(mess.split()) <= 1:
                continue
            elif len(mess.split()) > 1:
                print("TARGET:DELETE : " + mess.split()[1].strip())
                row = [file.name, "TARGET:DELETE", i + 1, mess.split()[1].strip()]
                report.append(row)

                
time.sleep(5)
clear_output()

pd.DataFrame(report, columns=target_column)

# Target Table

## Insert Into


## 剩餘的也都一樣


[目錄](#目錄)

In [ ]:
# Report list
report = []

for file in files:

    print('======= 處理 {}========'.format(file.name))
    
    sql_line = readline_file_with_encode(file)
    
    # Step02 - Step03 SQL 邏輯檢查
    for i, line in enumerate(sql_line):
        
        # Target 邏輯盤點
        
        # 1. Drop Table
        if re.search(r'\bDROP TABLE IF EXISTS\b', line, flags=re.I):
            # 去除前置空白 以及 DROP TABLE IF EXISTS 前的多餘字串
            mess = line[line.find("DROP TABLE IF EXISTS"):].replace(";","").lstrip()
            
            if len(mess.split()) <= 4:
                # 以空白切割字串 
                # 若只有 DROP TABLE IF EXISTS 那就只有 4 個字 故長度為 4
                continue
                
            elif len(mess.split()) > 4:
                print("TARGET:DROP TABLE IF EXISTS : " + mess.split()[4].strip())
                # 第 5 個字詞位置 為 Table Name
                row = [file.name, "TARGET:DROP TABLE IF EXISTS", i + 1, mess.split()[4].strip()]
                report.append(row)
                
            else:
                continue
                
        elif re.search(r'\bDROP TABLE\b', line, flags=re.I):
            mess = line[line.find("DROP TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:DROP TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:DROP TABLE", i + 1, mess.split()[2].strip()]
                report.append(row)
            else:
                continue
                  
        # 2. Create Table
        elif re.search(r'\bCREATE TABLE\b', line, flags=re.I):
            mess = line[line.find("CREATE TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:CREATE TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:CREATE TABLE", i + 1, mess.split()[2].strip()]
                report.append(row)
            else:
                continue

        # 3. Delete Table
        # 要注意的是 DELETE FROM 要先判斷
        elif re.search(r'\bDELETE FROM\b', line, flags=re.I):
            mess = line[line.find("DELETE FROM"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:DELETE FROM : " + mess.split()[2].strip())
                row = [file.name, "TARGET:DELETE FROM", i + 1, mess.split()[2].strip()]
                report.append(row)
        
        #  之後才判斷 DELETE TABLE NOT FROM (TD Only)
        elif re.search(r'\bDELETE\b', line):
            mess = line[line.find("DELETE"):].replace(";","").lstrip()
            if len(mess.split()) <= 1:
                continue
            elif len(mess.split()) > 1:
                print("TARGET:DELETE : " + mess.split()[1].strip())
                row = [file.name, "TARGET:DELETE", i + 1, mess.split()[1].strip()]
                report.append(row)

        # 4. Insert & 剩餘其他
        elif re.search(r'\bINSERT INTO\b', line, flags=re.I):
            mess = line[line.find("INSERT INTO"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:INSERT INTO : " + mess.split()[2])
                row = [file.name, "TARGET:INSERT INTO", i + 1, mess.split()[2].strip()]
                report.append(row)
            else:
                continue
                
        # TRUNCATE TABLE
        elif re.search(r'\bTRUNCATE TABLE\b', line, flags=re.I):
            mess = line[line.find("TRUNCATE TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:TRUNCATE TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:TRUNCATE TABLE", i + 1, mess.split()[2].strip()]
                report.append(row)

            else:
                continue
                
        # REPLACE VIEW
        elif re.search(r'\bREPLACE VIEW\b', line, flags=re.I):
            mess = line[line.find("REPLACE VIEW"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:REPLACE VIEW : " + mess.split()[2].strip())
                row = [file.name, "TARGET:REPLACE VIEW", i + 1, mess.split()[2].strip()]
                report.append(row)

            else:
                continue

                
time.sleep(5)
clear_output()

pd.DataFrame(report, columns=target_column)

# 以上 Target  部分


# 以下 Source  部分


## 暫時區分兩邊


## 之後再合併

# Source 部分


# From

[目錄](#目錄)

In [ ]:
report = []

for file in files:

    print('======= 處理 {}========'.format(file.name))
    
    sql_line = readline_file_with_encode(file)
    
    # Step02 - Step03 SQL 邏輯檢查
    for i, line in enumerate(sql_line):
        
        # Source 邏輯盤點
        # FROM
        if re.search(r'\bFROM\b', line, flags=re.I):

            if re.search(r"""['"].*?from.*?['"]""", line, flags=re.I):
                # 判斷 FROM 是否出現在 ' ' 之中 或 " " 之中
                continue
            
            start_regex_index = re.search(r'\bFROM\b', line, flags=re.I).start()
            mess = line[start_regex_index:].replace(";","").lstrip()
            if len(mess.split()) == 1:
                # 單行只寫一個 FROM  --> 跳過
                continue
            elif mess.split()[1].startswith('('):
                # FROM SubQuery  --> 跳過
                continue         
    #       FROM 內的 inner join 處理
            elif len(mess.split(",")) > 1:
                for element in mess.split(","):
                    # 多個 INNER JOIN Table 判斷
                    element = element.lstrip()
                    if element:
                        element = element.replace("FROM","").lstrip()
                        ele_check = element.split()
                        if len(ele_check) == 0:
                            continue
                        element = element.split()[0].replace(";","").lstrip()
                        print("SOURCE:FROM : " + element.strip())
                        row = [file.name, i, "SOURCE:FROM" + 1, element.strip()]
                        report.append(row)
            elif len(mess.split()) > 1:
                print("SOURCE:FROM : " + mess.split()[1].strip())
                row = [file.name, "SOURCE:FROM", i + 1, mess.split()[1].strip()]
                report.append(row)
            else:
                continue
                
time.sleep(5)
clear_output()
# 帶入 Source Columns
pd.DataFrame(report, columns=source_column)

# Source 部分


# Join

[目錄](#目錄)

In [ ]:
report = []

for file in files:

    sql_line = readline_file_with_encode(file)
    
    # Step02 - Step03 SQL 邏輯檢查
    for i, line in enumerate(sql_line):
        
        # Source 邏輯盤點
        # FROM
        if re.search(r'\bFROM\b', line, flags=re.I):

            if re.search(r"""['"].*?from.*?['"]""", line, flags=re.I):
                # 判斷 FROM 是否出現在 ' ' 之中 或 " " 之中
                continue
            
            start_regex_index = re.search(r'\bFROM\b', line, flags=re.I).start()
            mess = line[start_regex_index:].replace(";","").lstrip()
            if len(mess.split()) == 1:
                # 單行只寫一個 FROM  --> 跳過
                continue
            elif mess.split()[1].startswith('('):
                # FROM SubQuery  --> 跳過
                continue         
    #       FROM 內的 inner join 處理
            elif len(mess.split(",")) > 1:
                for element in mess.split(","):
                    # 多個 INNER JOIN Table 判斷
                    element = element.lstrip()
                    if element:
                        element = element.replace("FROM","").lstrip()
                        ele_check = element.split()
                        if len(ele_check) == 0:
                            continue
                        element = element.split()[0].replace(";","").lstrip()
                        print("SOURCE:FROM : " + element.strip())
                        row = [file.name, i, "SOURCE:FROM" + 1, element.strip()]
                        report.append(row)
            elif len(mess.split()) > 1:
                print("SOURCE:FROM : " + mess.split()[1].strip())
                row = [file.name, "SOURCE:FROM", i + 1, mess.split()[1].strip()]
                report.append(row)
            else:
                continue
                
        # JOIN
        elif re.search(r'\bJOIN\b', line, flags=re.I):
#             print('Line : {}'.format(i + 1))
            start_regex_index = re.search(r'\bJOIN\b', line, flags=re.I).start()
            mess = line[start_regex_index:].replace(";","").lstrip()
            if len(mess.split()) == 1:
                continue
            elif mess.split()[1].startswith('('):
                continue
            elif len(mess.split()) > 1:
                print("SOURCE:JOIN : " + mess.split()[1].strip())
                row = [file.name, "SOURCE:JOIN", i + 1, mess.split()[1].strip()]
                report.append(row)
            else:
                continue

time.sleep(5)
clear_output()
# 帶入 Target Columns
pd.DataFrame(report, columns=target_column)                


# 組合


## Target & Source 組合前 先想幾個資料呈現問題


### &emsp; 1. 欄位變化 ？



### &emsp; 2. Target & Source Table 數量不一致？



### &emsp; 3. 重複的 TableName



- 欄位應該變成 

|  檔案名稱  |  Source Operator  | Source Line  | Source Table | Target Operator | Target Line | Target Table
|  ----  | ----  |  ----  | ---- | ---- | ---- | ---- 


- 資料呈現

1. Source Table 較多時

|  檔案名稱  |  Source Operator  | Source Line  | Source Table | Target Operator | Target Line | Target Table
|  ----  | ----  |  ----  | ---- | ---- | ---- | ---- 
| File1  | FROM | 4 | PDATA.TABLENAME | INSERT | 3 | PMART.TABLENAME 
| File1  | JOIN | 9 | PMART.TABLENAME | DROP | 14 | PTEMP.TABLENAME
| File1  | FROM | 13 | PDATA.TABLENAME |  |  |  



2. Target Table 較多時

|  檔案名稱  |  Source Operator  | Source Line  | Source Table | Target Operator | Target Line | Target Table
|  ----  | ----  |  ----  | ---- | ---- | ---- | ---- 
| File1  | FROM | 4 | PDATA.TABLENAME | INSERT | 3 | PMART.TABLENAME 
| File1  | JOIN | 9 | PMART.TABLENAME | DROP | 14 | PTEMP.TABLENAME
| File1  |  |  |  | DROP | 16 | PVIEW.VIEW 
| File1  |  |  |  | REPLACE VIEW | 18 | PVIEW.VIEW 


[目錄](#目錄)


## 承上


### 一行資料中 應該包含 

`檔案名稱	Source Operator	Source Line	Source Table	Target Operator	Target Line	Target Table`


### 回去觀察  （ 取 找出 JOIN 部分來看 ）

```python
# JOIN
        elif re.search(r'\bJOIN\b', line, flags=re.I):
#             print('Line : {}'.format(i + 1))
            start_regex_index = re.search(r'\bJOIN\b', line, flags=re.I).start()
            mess = line[start_regex_index:].replace(";","").lstrip()
            if len(mess.split()) == 1:
                continue
            elif mess.split()[1].startswith('('):
                continue
            elif len(mess.split()) > 1:
                print("SOURCE:JOIN : " + mess.split()[1].strip())
                row = [file.name, "SOURCE:JOIN", i + 1, mess.split()[1].strip()]
                report.append(row)
            else:
                continue

```


### 我們只在找到 Source / Target 時 產生 一行資料


## --> *之後重新組合 Source / Target*


- 拆分 Report List  

&emsp; &emsp; 1. Source List


&emsp; &emsp; 2. Target List


- 重新組合 Source / Target  --> Report List



[目錄](#目錄)

In [ ]:

# 先組合 並拆分成 Source / Target

def extract_table(file):
    
    # 拆分 target / source
    
    target_list = []
    source_list = []
    
    sql_line = readline_file_with_encode(file)
    
    for i, line in enumerate(sql_line):
        # Target 複製下來 但 report_list 改成 target_list
        
        # Target 邏輯盤點
        
        # 1. Drop Table
        if re.search(r'\bDROP TABLE IF EXISTS\b', line, flags=re.I):
            # 去除前置空白 以及 DROP TABLE IF EXISTS 前的多餘字串
            mess = line[line.find("DROP TABLE IF EXISTS"):].replace(";","").lstrip()
            
            if len(mess.split()) <= 4:
                # 以空白切割字串 
                # 若只有 DROP TABLE IF EXISTS 那就只有 4 個字 故長度為 4
                continue
                
            elif len(mess.split()) > 4:
                print("TARGET:DROP TABLE IF EXISTS : " + mess.split()[4].strip())
                # 第 5 個字詞位置 為 Table Name
                row = [file.name, "TARGET:DROP TABLE IF EXISTS", i + 1, mess.split()[4].strip()]
                target_list.append(row)
                
            else:
                continue
                
        elif re.search(r'\bDROP TABLE\b', line, flags=re.I):
            mess = line[line.find("DROP TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:DROP TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:DROP TABLE", i + 1, mess.split()[2].strip()]
                target_list.append(row)
            else:
                continue
                  
        # 2. Create Table
        elif re.search(r'\bCREATE TABLE\b', line, flags=re.I):
            mess = line[line.find("CREATE TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:CREATE TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:CREATE TABLE", i + 1, mess.split()[2].strip()]
                target_list.append(row)
            else:
                continue

        # 3. Delete Table
        # 要注意的是 DELETE FROM 要先判斷
        elif re.search(r'\bDELETE FROM\b', line, flags=re.I):
            mess = line[line.find("DELETE FROM"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:DELETE FROM : " + mess.split()[2].strip())
                row = [file.name, "TARGET:DELETE FROM", i + 1, mess.split()[2].strip()]
                target_list.append(row)
        
        #  之後才判斷 DELETE TABLE NOT FROM (TD Only)
        elif re.search(r'\bDELETE\b', line):
            mess = line[line.find("DELETE"):].replace(";","").lstrip()
            if len(mess.split()) <= 1:
                continue
            elif len(mess.split()) > 1:
                print("TARGET:DELETE : " + mess.split()[1].strip())
                row = [file.name, "TARGET:DELETE", i + 1, mess.split()[1].strip()]
                target_list.append(row)

        # 4. Insert & 剩餘其他
        elif re.search(r'\bINSERT INTO\b', line, flags=re.I):
            mess = line[line.find("INSERT INTO"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:INSERT INTO : " + mess.split()[2])
                row = [file.name, "TARGET:INSERT INTO", i + 1, mess.split()[2].strip()]
                target_list.append(row)
            else:
                continue
                
        # TRUNCATE TABLE
        elif re.search(r'\bTRUNCATE TABLE\b', line, flags=re.I):
            mess = line[line.find("TRUNCATE TABLE"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:TRUNCATE TABLE : " + mess.split()[2].strip())
                row = [file.name, "TARGET:TRUNCATE TABLE", i + 1, mess.split()[2].strip()]
                target_list.append(row)

            else:
                continue
                
        # REPLACE VIEW
        elif re.search(r'\bREPLACE VIEW\b', line, flags=re.I):
            mess = line[line.find("REPLACE VIEW"):].replace(";","").lstrip()
            if len(mess.split()) <= 2:
                continue
            elif len(mess.split()) > 2:
                print("TARGET:REPLACE VIEW : " + mess.split()[2].strip())
                row = [file.name, "TARGET:REPLACE VIEW", i + 1, mess.split()[2].strip()]
                target_list.append(row)

            else:
                continue
                
        # Source 部分 複製下來 report 改成 source_list
        # Source 邏輯盤點
        # FROM
        if re.search(r'\bFROM\b', line, flags=re.I):

            if re.search(r"""['"].*?from.*?['"]""", line, flags=re.I):
                # 判斷 FROM 是否出現在 ' ' 之中 或 " " 之中
                continue
            
            start_regex_index = re.search(r'\bFROM\b', line, flags=re.I).start()
            mess = line[start_regex_index:].replace(";","").lstrip()
            if len(mess.split()) == 1:
                # 單行只寫一個 FROM  --> 跳過
                continue
            elif mess.split()[1].startswith('('):
                # FROM SubQuery  --> 跳過
                continue         
    #       FROM 內的 inner join 處理
            elif len(mess.split(",")) > 1:
                for element in mess.split(","):
                    # 多個 INNER JOIN Table 判斷
                    element = element.lstrip()
                    if element:
                        element = element.replace("FROM","").lstrip()
                        ele_check = element.split()
                        if len(ele_check) == 0:
                            continue
                        element = element.split()[0].replace(";","").lstrip()
                        print("SOURCE:FROM : " + element.strip())
                        row = [file.name, i, "SOURCE:FROM" + 1, element.strip()]
                        source_list.append(row)
            elif len(mess.split()) > 1:
                print("SOURCE:FROM : " + mess.split()[1].strip())
                row = [file.name, "SOURCE:FROM", i + 1, mess.split()[1].strip()]
                source_list.append(row)
            else:
                continue
                
        # JOIN
        elif re.search(r'\bJOIN\b', line, flags=re.I):
#             print('Line : {}'.format(i + 1))
            start_regex_index = re.search(r'\bJOIN\b', line, flags=re.I).start()
            mess = line[start_regex_index:].replace(";","").lstrip()
            if len(mess.split()) == 1:
                continue
            elif mess.split()[1].startswith('('):
                continue
            elif len(mess.split()) > 1:
                print("SOURCE:JOIN : " + mess.split()[1].strip())
                row = [file.name, "SOURCE:JOIN", i + 1, mess.split()[1].strip()]
                source_list.append(row)
            else:
                continue
    
    # 同時 return Target / Source
    return target_list, source_list
        
        
        
    


In [ ]:
# 測試一下

target, source = extract_table(files[36])


time.sleep(5)
clear_output()
print('============= Target 部分')

display(pd.DataFrame(target, columns=target_column))

print('\n')
print('============= Source 部分')

display(pd.DataFrame(source, columns=source_column))




# 重新整理 Report List


## 動腦時間


## 目標 ： 2 個 row 合成一個

<br>

|  檔案名稱  |  Source Operator  | Source Line  | Source Table | Target Operator | Target Line | Target Table
|  ----  | ----  |  ----  | ---- | ---- | ---- | ----
| File1  | FROM | 4 | PDATA.TABLENAME  
| File1  |  |  |  | DROP | 3 | PTEMP.TABLENAME


<br>
 <div align="center">
    <span style="color:red;">直接合併的 報表 ↑ </span>
</div>
<br>

<br>
 <div align="center">
    <span style="color:red;">我腦袋中的 報表 ↓ </span>
</div>
<br>

|  檔案名稱  |  Source Operator  | Source Line  | Source Table | Target Operator | Target Line | Target Table
|  ----  | ----  |  ----  | ---- | ---- | ---- | ----
| File1  | FROM | 4 | PDATA.TABLENAME | DROP | 3 | PTEMP.TABLENAME



[目錄](#目錄)


# 最快解 --> 同時遞迴Target / Source List 並重組 每個 row


- 使用 zip function  --> 可 __同時遞迴 「多組」 可遞迴的物件__ 

```python

a = [1, 5, 4, 3, 7]
b = [10, 6, 9, 11, 6]

for _a , _b in zip(a, b):
    
    print('a 是 {} , b 是 {}'.format(_a, _b))

```

```
a 是 1 , b 是 10
a 是 5 , b 是 6
a 是 4 , b 是 9
a 是 3 , b 是 11
a 是 7 , b 是 6
```

## 但若 Target List / Source List 長度不一

- 使用 zip_longest function 

- 長度較短者 則回傳 None


```python

from itertool import zip_longest

a = [1, 5, 4, 3, 7]
b = [10, 6, 9, 11, 6, 2, 4, 7]

for _a , _b in zip_longest(a, b):
    
    print('a 是 {} , b 是 {}'.format(_a, _b))

```

```
a 是 1 , b 是 10
a 是 5 , b 是 6
a 是 4 , b 是 9
a 是 3 , b 是 11
a 是 7 , b 是 6
a 是 None , b 是 2
a 是 None , b 是 4
a 是 None , b 是 7

```


[目錄](#目錄)

In [ ]:
report_columns = [
    '檔案名稱',
    'Source Operator',
    'Source Line',
    'Source Table',
    'Target Operator',
    'Target Line',
    'Target Table',
]

In [ ]:
report_list = []

for _target, _source in zip_longest(target, source):
    
    row = [files[28].name]
    if _source:
        # extend 可直接做 list 合併
        row.extend([_source[1], _source[2], _source[3]])
    
    # 若 source 較少時 會回傳 None
    else:
        # 都是 None
        row.extend([_source, _source, _source])
        
    if _target:
        row.extend([_target[1], _target[2], _target[3]])
    
    # 若 target 較少時 會回傳 None
    else:
        row.extend([_target, _target, _target])
        
    report_list.append(row)
        
        
pd.DataFrame(report_list, columns=report_columns)
        
        




# 去重複


## 重複 Table 的定義應為 （若不盤點 DDL的行為 如： Drop or Create）


### &emsp; 1. 單個 檔案內重複的 Source TableName

### &emsp; 2. 單個 檔案內重複的 Target TableName


## 這邊我用 pandas Drop Duplicate 來解


- 比較快 容易寫

[目錄](#目錄)


In [ ]:
target, source = extract_table(files[28])

# 去除重複

time.sleep(3)
clear_output()

print('去除重複前： ')

display(pd.DataFrame(target, columns=target_column))

print('\n')
print('去除重複後： ')

# 預設是留下第一筆
# 若要盤點 DDL 行為 則把 Operator 加入 subset 中
display(pd.DataFrame(target, columns=target_column).drop_duplicates(subset=['Target_Table']))

# 最後整合

[目錄](#目錄)

In [ ]:

report_list = []

target, source = extract_table(files[28])


# 去重複 - Target

df = pd.DataFrame(target, columns=target_column).drop_duplicates(subset=['Target_Table'])

# 重新組合 去重後的 Target List
re_target = []

# pandas.DataFrame.iterrows --> 回傳 index 跟 row
for index, row in df.iterrows():
    
    re_target.append(row)
    
# 去重複 - Source

df = pd.DataFrame(source, columns=source_column).drop_duplicates(subset=['Source_Table'])

re_source = []

for index, row in df.iterrows():
    re_source.append(row)

# 盤點部分
for _target, _source in zip_longest(re_target, re_source):
    
    row = [files[9].name]
    if _source is not None:
        # extend 可直接做 list 合併
        row.extend([_source[1], _source[2], _source[3]])
    
    # 若 source 較少時 會回傳 None
    else:
        # 都是 None
        row.extend([_source, _source, _source])
        
    if _target is not None:
        row.extend([_target[1], _target[2], _target[3]])
    
    # 若 target 較少時 會回傳 None
    else:
        row.extend([_target, _target, _target])
        
    report_list.append(row)
        

time.sleep(3)
clear_output()        

pd.DataFrame(report_list, columns=report_columns)
        


# 大量做


## 能做一個 File 大量做就容易了


### 全塞進 for loop

[目錄](#目錄)

In [ ]:
report_list = []

for file in files:
    
    print('=============== 現在處理 {}'.format(file.name))

    target, source = extract_table(file)


    # 去重複 - Target

    df = pd.DataFrame(target, columns=target_column).drop_duplicates(subset=['Target_Table'])

    # 重新組合 去重後的 row 
    re_target = []

    # pandas.DataFrame.iterrows --> 回傳 index 跟 row
    for index, row in df.iterrows():

        re_target.append(row)

    # 去重複 - Source

    df = pd.DataFrame(source, columns=source_column).drop_duplicates(subset=['Source_Table'])

    re_source = []

    for index, row in df.iterrows():
        re_source.append(row)

    for _target, _source in zip_longest(re_target, re_source):

        row = [file.name]
        if _source is not None:
            # extend 可直接做 list 合併
            row.extend([_source[1], _source[2], _source[3]])

        # 若 source 較少時 會回傳 None
        else:
            # 都是 None
            row.extend([_source, _source, _source])

        if _target is not None:
            row.extend([_target[1], _target[2], _target[3]])

        # 若 target 較少時 會回傳 None
        else:
            row.extend([_target, _target, _target])

        report_list.append(row)
        

time.sleep(3)
clear_output()        

pd.DataFrame(report_list, columns=report_columns)

In [ ]:
pd.DataFrame(report_list, columns=report_columns).to_excel('TEST.xlsx', index=False)